# TP Intelligence Artificielle - Recherche Arborescente Non Informée

# **Partie 0 : Visualisation des états**

## Voici une fonction pour visualiser les états. Nous l'utiliserons plus tard.

In [1]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# **Partie 1 : Modélisation**

### Nous allons créer deux classes pour modeliser le taquin en espace d'états.

1. **Taquin** : Cette classe représente le problème du jeu du Taquin. Elle contient :

  * Attributs:

      * initial_state : L'état initial du Taquin, représenté par une liste de listes. Chaque sous-liste représente une ligne du Taquin, et chaque élément de la sous-liste représente une tuile. La tuile vide est représentée par le chiffre 0.
      * goal_state : L'état but du Taquin, représenté de la même manière que l'état initial.
      * size : La taille du Taquin (par exemple, 3 pour un Taquin 3x3, 4 pour un Taquin 4x4).

  * Méthodes:

      * actions(state) : Cette méthode prend un état du Taquin en entrée et retourne une liste des actions possibles à partir de cet état. Les actions possibles sont "haut", "bas", "gauche" et "droite", représentant les mouvements possibles de la tuile vide.
      * result(state, action) : Cette méthode prend un état et une action en entrée et retourne le nouvel état du Taquin après avoir appliqué l'action à l'état.
      * is_goal(state) : Cette méthode prend un état en entrée et retourne True si cet état est l'état but, False sinon.
      * cost(state, action) : Cette méthode retourne le coût de l'application d'une action à un état donné. Dans le cas du Taquin, le coût est généralement constant et égal à 1 pour chaque action.
  
2.   **Node** :  Cette classe représentera un nœud dans l'arbre de recherche. Ces attributs sont :

  * state : L'état représenté par ce nœud.
  * parent : Un pointeur vers le nœud parent (None pour le nœud racine).
  * action : L'action qui a conduit à ce nœud à partir du nœud parent (None pour le nœud racine).
  * path_cost : Le coût total du chemin depuis le nœud racine jusqu'à ce nœud (facultatif, pour les algorithmes avec des considérations de coût).
  * depth : La profondeur de ce nœud dans l'arbre (facultatif, pour la recherche en profondeur limitée).



--------------------------------------------------------------------------------
## **1.1 Classe Taquin**
--------------------------------------------------------------------------------

In [4]:
class Taquin:
    """
    表示Taquin拼图问题的类。
    Taquin问题是经典的滑动拼图问题，目标是通过移动拼图块达到给定的目标状态。
    """

    def __init__(self, initial_state, goal_state, size):
        """
        初始化Taquin问题实例。

        参数：
        initial_state: 初始状态，以二维列表表示拼图块的布局。
        goal_state: 目标状态，同样以二维列表表示。
        size: 拼图大小（例如：3表示3x3拼图）。
        """
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """
        根据给定状态返回所有可能的动作（即空白拼图块可以移动的方向）。

        参数：
        state: 当前的拼图状态，以二维列表表示。

        返回：
        一个列表，包含所有可行的动作（字符串形式："up"、"down"、"left"、"right"）。
        """
        # 首先找到空白块（用0表示）的坐标
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # 根据空白块位置，确定可能的移动方向
        possible_actions = []

        if row > 0:  # 空白块不在第一行，可以向上移动
            possible_actions.append("up")
        if row < self.size - 1:  # 空白块不在最后一行，可以向下移动
            possible_actions.append("down")
        if col > 0:  # 空白块不在第一列，可以向左移动
            possible_actions.append("left")
        if col < self.size - 1:  # 空白块不在最后一列，可以向右移动
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """
        对给定状态执行动作后返回产生的新状态。

        参数：
        state: 当前的拼图状态，以二维列表表示。
        action: 执行的动作（"up"、"down"、"left"、"right"）。

        返回：
        一个新状态（二维列表），表示执行该动作后的拼图布局。
        """
        # 创建状态的副本，以避免直接修改原始状态
        new_state = [list(row) for row in state]

        # 找到当前空白块的坐标
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # 根据动作移动空白块，并与相邻块交换位置
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        """
        检测给定状态是否为目标状态。

        参数：
        state: 待检测的拼图状态。

        返回：
        True（是目标状态）或False（不是目标状态）。
        """
        return state == self.goal_state  # 直接比较当前状态和目标状态

    def cost(self, state, action):
        """
        返回执行某个动作的代价。
        一般Taquin问题的代价都是统一为1（每个动作成本相同）。

        参数：
        state: 当前状态。
        action: 将要执行的动作。

        返回：
        整数值1，表示移动拼图块的代价。
        """
        return 1


## **Exercise 1**
1. Créez un jeu de taquin 4x4 avec un état initial et un état objectif, puis visualisez ces deux états.
2. Identifiez les actions possibles à partir de l'état initial.
3. Appliquez une des actions possibles et visualisez le nouvel état.


In [3]:
# 定义拼图问题的初始状态和目标状态：
initial_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 0, 15]  # 0代表空白方块，当前位于第4行第3列
]

# 输出初始状态
print("Initial state:")
visualize_state(initial_state)  # visualize_state()函数用来可视化显示拼图状态

# 设定拼图目标状态（最终希望达到的状态）
goal_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]  # 目标中空白方块在右下角（第4行第4列）
]

# 输出目标状态
print("Goal state:")
visualize_state(goal_state)  # 可视化目标状态

# 根据定义好的初始状态和目标状态创建Taquin问题实例
problem = Taquin(initial_state, goal_state, len(initial_state[0]))
# len(initial_state[0]) = 4，即拼图为4x4的大小

# 调用actions()方法，获得从初始状态能进行的所有可能动作
actions = problem.actions(initial_state)
print("Possible actions:", actions)
# 输出可能的动作列表，如["up", "down", "left", "right"]

# 尝试从当前初始状态执行其中一个动作，例如 'up'
new_state = problem.result(initial_state, 'up')
print("Applied action `up`. The new state is")
visualize_state(new_state)  # 可视化执行动作后的新状态

Initial state:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,,15


Goal state:


1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,


Possible actions: ['up', 'left', 'right']
Applied action `up`. The new state is


1,2,3,4
5,6,7,8
9,10,,12
13,14,11,15


--------------------------------------------------------------------------------
## **1.2 Classe Node**
--------------------------------------------------------------------------------

In [9]:
class Node:
    """
    表示搜索树（search tree）中的一个节点（Node）。

    Node类用于记录当前的状态、父节点、执行的动作、到达当前节点的路径代价、以及当前节点的深度。
    在搜索问题（如拼图Taquin问题）中，节点结构帮助我们追踪状态之间的转移过程，最终重建从初始状态到目标状态的路径。

    关键方法说明:
    - __init__: 初始化节点
    - __repr__: 返回节点的字符串表示（注释掉）
    - __lt__: 定义节点间的比较规则（注释掉）
    - expand: 从当前节点扩展，生成所有可能的子节点
    - child_node: 为指定动作生成子节点
    - solution: 提供从初始节点到当前节点所需执行的动作序列
    - path: 返回从根节点到当前节点的完整路径
    """

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """
        初始化节点实例：

        参数：
        - state: 节点所代表的当前状态
        - parent: 父节点（默认为None，若为根节点则无父节点）
        - action: 从父节点到当前节点执行的动作
        - path_cost: 从初始节点到达当前节点的路径总代价（默认为0）

        属性：
        - depth: 节点在搜索树中的深度（根节点深度为0）
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    # 注释掉的__repr__方法，可以根据需求重新打开，用于调试时输出节点信息
    # def __repr__(self):
    #     return "<Node {}>".format(self.state)

    # 注释掉的__lt__方法，定义节点之间的比较方式（例如用于优先队列排序）
    # def __lt__(self, other):
    #     return self.state < other.state

    def expand(self, problem):
        """
        从当前节点扩展，生成所有可达的子节点。

        参数：
        - problem: 表示问题的实例，包含actions()、result()、cost()方法。

        返回：
        - 一个节点列表，每个节点代表从当前节点执行一个合法动作后的新状态。
        """
        return [
            self.child_node(problem, action)  # 针对每个可能动作，创建子节点
            for action in problem.actions(self.state)  # 获取所有可能的动作
        ]

    def child_node(self, problem, action):
        """
        创建并返回通过执行特定动作而得到的子节点。

        参数：
        - problem: 问题实例，提供result()和cost()方法
        - action: 要执行的具体动作

        返回：
        - 执行动作后的新节点（子节点）
        """
        next_state = problem.result(self.state, action)  # 根据action获取新状态
        next_node = Node(
            state=next_state,            # 新状态
            parent=self,                 # 当前节点为父节点
            action=action,               # 执行的动作
            path_cost=self.path_cost + problem.cost(self.state, action)  # 累计路径成本
        )
        return next_node

    def solution(self):
        """
        提供从根节点（初始状态）到当前节点所需执行的动作序列。

        返回：
        - 一个动作列表，从初始状态到当前节点的具体动作（不含初始节点动作为None）
        """
        return [node.action for node in self.path()[1:]]  # 跳过根节点（action=None）

    def path(self):
        """
        返回从根节点到当前节点的完整节点路径列表。

        返回：
        - 节点列表，顺序为从初始状态节点到当前节点。
        """
        node, path_back = self, []  # 从当前节点开始向上回溯
        while node:
            path_back.append(node)  # 逐个向上添加节点至列表
            node = node.parent      # 回溯至父节点
        return list(reversed(path_back))  # 将列表反转为初始节点到当前节点的顺序


## **Exercise 2**
1. Créez un jeu de taquin 3x3 avec un état initial et un état objectif.
2. Créez un nœud représentant l'état initial du jeu.
3. Visualisez les enfants de l'état initial.
4. Pour chaque enfant, imprimez :
    * L'action à effectuer pour passer de l'état initial à cet enfant.
    * Le coût associé à cette action.
5. Répétez toutes les étapes pour un jeu de taquin 4x4.


In [10]:
print('------------- Taquin 3x3 -------------')

# 定义初始状态和目标状态 (3x3拼图)
initial_state = [[1, 2, 3],
                 [4, 0, 5],   # 空白方块(0)位于中间位置
                 [7, 8, 6]]

goal_state = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]     # 空白方块(0)最终位置为右下角

# 创建Taquin实例对象
problem = Taquin(initial_state, goal_state, len(initial_state[0]))  # 拼图尺寸为3x3

# 使用初始状态创建搜索树的根节点
initial_node = Node(initial_state)

# 可视化输出初始状态
print("Initial state:")
visualize_state(initial_node.state)

# 从根节点扩展得到所有子节点（即当前状态下所有可移动到的新状态）
children = initial_node.expand(problem)

# 显示所有子节点状态及相关信息
print("\nChildren nodes:")
for child in children:
    print(" ")
    visualize_state(child.state)                    # 可视化每个子节点的拼图状态
    print("Action to arrive to that node :", child.action)  # 显示为达到该状态采取的动作
    print("Path cost:", child.path_cost)            # 显示从初始状态到该状态的路径代价


------------- Taquin 3x3 -------------
Initial state:


1,2,3
4,,5
7,8,6



Children nodes:
 


1,,3
4,2,5
7,8,6


Action to arrive to that node : up
Path cost: 1
 


...

Action to arrive to that node : down
Path cost: 1
 


1,2,3
,4,5
7,8,6


Action to arrive to that node : left
Path cost: 1
 


1,2,3
4,5,
7,8,6


Action to arrive to that node : right
Path cost: 1


In [11]:
print('------------- Taquin 4x4 -------------')

# 定义初始状态和目标状态 (4x4拼图)
initial_state = [[1, 2, 3, 4],
                 [5, 6, 7, 8],
                 [9, 10, 0, 11],   # 空白方块(0)位于第三行第三列
                 [13, 14, 15, 12]]

goal_state = [[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 0]]     # 空白方块(0)最终位于右下角

# 创建Taquin实例对象
problem = Taquin(initial_state, goal_state, len(initial_state[0]))  # 拼图尺寸为4x4

# 使用初始状态创建搜索树的根节点
initial_node = Node(initial_state)

# 可视化输出初始状态
print("Initial state:")
visualize_state(initial_node.state)

# 从根节点扩展得到所有子节点
children = initial_node.expand(problem)

# 显示所有子节点状态及相关信息
print("\nChildren nodes:")
for child in children:
    print(" ")
    visualize_state(child.state)                     # 可视化每个子节点的拼图状态
    print("Action to arrive to that node :", child.action)  # 为达到此状态所采取的动作
    print("Path cost:", child.path_cost)             # 从初始状态到该状态的路径代价


------------- Taquin 4x4 -------------
Initial state:


1,2,3,4
5,6,7,8
9,10,,11
13,14,15,12



Children nodes:
 


1,2,3,4
5,6,,8
9,10,7,11
13,14,15,12


Action to arrive to that node : up
Path cost: 1
 


...

Action to arrive to that node : down
Path cost: 1
 


1,2,3,4
5,6,7,8
9,,10,11
13,14,15,12


Action to arrive to that node : left
Path cost: 1
 


1,2,3,4
5,6,7,8
9,10,11,
13,14,15,12


Action to arrive to that node : right
Path cost: 1


# **Partie 2 : BFS et DFS**

Dans cette 2ème partie nous allons coder les algorithmes de recherche arborescente que nous avons étudiés en cours.

## **2.1 Breadth First Search**

### **Exercice 3**

Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la frontière et l'ensemble des nœuds déjà explorés :
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* L'ensemble des nœuds explorés peut être une liste d'états.
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le premier élément de la liste.

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.

In [19]:
def breadth_first_search(problem, max_it=float('inf')):
    """
    广度优先搜索算法（Breadth-First Search，BFS）：
    从初始状态逐层展开，依次探索所有状态，直至找到目标状态或超过迭代限制。

    参数：
    - problem: 要解决的搜索问题实例，需提供initial_state, actions(), result(), is_goal()方法。
    - max_it: 最大迭代次数（默认无穷大，若设定则超过该次数停止搜索）。

    返回值：
    - 如果找到目标状态，则返回(goal_node, 迭代次数)；
    - 若未找到目标状态，则返回(None, 迭代次数)。
    """

    # 创建根节点，以初始状态初始化
    initial_node = Node(problem.initial_state)

    # 初始化frontier（待扩展节点列表），初始包含根节点
    frontier = [initial_node]

    # 已探索状态列表，防止重复探索
    explored = []

    # 迭代计数器
    it = 0

    # 开始搜索循环，直至frontier为空或达到最大迭代次数
    while frontier and it < max_it:
        it += 1

        # 从frontier取出第一个节点（队列先进先出原则，BFS核心）
        node = frontier.pop(0)

        # 将该节点状态标记为已探索
        explored.append(node.state)

        # 遍历当前节点所有可能动作
        for action in problem.actions(node.state):

            # 执行动作，获得子状态
            child_state = problem.result(node.state, action)

            # 如果子状态未被探索过
            if child_state not in explored:

                # 创建子节点，记录路径、动作和父节点
                child_node = Node(child_state, parent=node, action=action)

                # 检查子状态是否为目标状态
                if problem.is_goal(child_node.state):
                    # 若是目标状态，立即返回子节点及迭代次数
                    return child_node, it

                # 若子节点不在frontier中，则加入frontier等待后续扩展
                if child_node not in frontier:
                    frontier.append(child_node)

    # 若搜索结束未找到目标状态，返回None表示失败及迭代次数
    return None, it


### **Exercice 4**
1. Appliquez votre fonction BFS pour résoudre un Taquin 3x3 avec l'état initial
   [[1, 2, 3], [4, 5, 6], [0, 7, 8]].
   Imprimez :
    * Le nombre de nœuds explorés par l'algorithme.
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * Les états successifs du chemin (utilisez la fonction visualize_state).
3. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].

In [20]:
# 定义拼图的初始状态和目标状态 (3x3的拼图)
initial_state = [
    [1, 2, 3],
    [4, 5, 0],  # 空白方块(0)初始在第二行第三列
    [6, 7, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]   # 空白方块(0)目标位置在右下角
]

# 创建Taquin拼图问题实例
problem = Taquin(initial_state, goal_state, len(initial_state[0]))  # 拼图尺寸为3x3

# 使用广度优先搜索(BFS)算法尝试寻找拼图解法
solution_node, it = breadth_first_search(problem)

# 检测是否找到解决方案，并进行结果展示
if solution_node:
    print("Solution found in", it, "iterations")   # 显示找到解法所用的迭代次数
    print("Actions:", solution_node.solution())    # 显示解法所需的动作序列

    # 可视化并打印从初始状态到目标状态的完整路径（包括每一步）
    t = 1  # 用于标记路径上每个状态的序号
    for node in solution_node.path():
        print(f"{t}-th element of the path:")
        visualize_state(node.state)  # 可视化当前状态的拼图布局
        t += 1  # 递增状态序号
else:
    # 若未找到解决方案，输出提示信息
    print("No solution found.")


Solution found in 1846 iterations
Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
1-th element of the path:


1,2,3
4,5,
6,7,8


2-th element of the path:


1,2,3
4,5,8
6,7,


3-th element of the path:


...

1,2,3
4,5,6
,7,8


13-th element of the path:


1,2,3
4,5,6
7,,8


14-th element of the path:


1,2,3
4,5,6
7,8,


## **2.2 Depth First Search**

### **Exercice 5**
Créer une fonction DFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la frontière et l'ensemble des nœuds déjà explorés :
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* L'ensemble des nœuds explorés peut être une liste d'états.
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le dernier élément de la liste.

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.



In [22]:
def depth_first_search(problem, max_it=float('inf')):
    """
    深度优先搜索算法（Depth-First Search, DFS）:
    沿一条路径深入搜索，直到无法继续前进或达到目标，再回溯搜索其他路径。

    参数:
    - problem: 待解决的搜索问题实例（提供initial_state、actions()、result()、is_goal()方法）
    - max_it: 最大允许的迭代次数，防止无限搜索（默认无穷大）

    返回值:
    - 若找到目标状态，返回(goal_node, 迭代次数)
    - 若未找到目标状态，返回(None, 迭代次数)
    """

    # 从初始状态创建根节点
    initial_node = Node(problem.initial_state)

    # frontier用于记录待扩展节点（使用列表模拟栈，DFS核心特点为后进先出(LIFO)）
    frontier = [initial_node]

    # explored列表用于记录已经探索过的状态，避免重复探索
    explored = []

    # 迭代次数计数器
    it = 0

    # 搜索循环，持续到frontier为空或达到最大迭代次数
    while frontier and it < max_it:
        # 每次迭代增加计数器
        it += 1

        # 从frontier取出最后一个节点（DFS采用后进先出原则）
        node = frontier.pop(-1)

        # 将当前节点的状态标记为已探索
        explored.append(node.state)

        # 遍历当前节点所有可能的动作（下一步可以走的路径）
        for action in problem.actions(node.state):

            # 执行动作获得新的子状态
            child_state = problem.result(node.state, action)

            # 若子状态未被探索过（避免循环）
            if child_state not in explored:

                # 创建新的子节点
                child_node = Node(child_state, parent=node, action=action)

                # 若子节点状态即为目标状态，搜索成功，返回结果
                if problem.is_goal(child_node.state):
                    return child_node, it  # 返回找到的目标节点及当前迭代次数

                # 如果子节点尚未在frontier中，添加进frontier等待扩展
                if child_node not in frontier:
                    frontier.append(child_node)

    # 若搜索结束仍未找到目标，返回None以及迭代次数
    return None, it


### **Exercise 6**
1. Appliquez votre fonction DFS pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]]. Imprimez :
    * Le nombre de nœuds explorés par l'algorithme.
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * Les états successifs du chemin (utilisez la fonction visualize_state).
2. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
3. Qu'observez-vous en comparant cet algorithme avec BFS ?

In [23]:
# 定义拼图的初始状态（3x3拼图）
initial_state = [
    [1, 2, 3],
    [4, 5, 0],   # 空白方块(0)初始位于第2行第3列
    [6, 7, 8]
]

# 定义拼图的目标状态（拼图完成时的状态）
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]    # 空白方块(0)目标位于第3行第3列（右下角）
]

# 根据初始与目标状态创建一个拼图问题实例
problem = Taquin(initial_state, goal_state, len(initial_state[0]))  # 拼图尺寸为3x3

# 定义DFS算法允许的最大搜索迭代次数（防止无限循环）
max_it = 10000

# 执行深度优先搜索（DFS），尝试找到拼图从初始状态到目标状态的解法
solution_node, it = depth_first_search(problem, max_it)

# 检查DFS算法是否成功找到解法
if solution_node:
    print("Solution found in", it, 'iterations')  # 显示找到解所需的迭代次数
    print("Actions:", solution_node.solution())   # 显示解法对应的动作序列

    # 逐步显示从初始状态到目标状态的所有状态变化过程
    t = 1  # 用于记录路径中的步骤序号
    for node in solution_node.path():
        print(t, '-th element of the path')
        visualize_state(node.state)   # 可视化打印拼图当前的状态
        t = t + 1  # 增加步骤计数器
else:
    # 如果没有找到解法，则打印提示信息
    print("No solution found.")


No solution found.


## **2.3 Comparaison empirique de DFS et BFS**

Nous allons comparer la performance des deux algorithmes dans plusieurs jeux de Taquin 3x3.

### **Exercice 7**
1. Écrivez une fonction compare_algorithms qui prend en entrée :
    * Une liste d’états initiaux.
    * Un état objectif.
    * Une liste d’algorithmes.
2. La fonction doit résoudre chaque problème en exécutant chaque algorithme. Assurez-vous de fixer des budgets pour éviter que les algorithmes ne prennent trop de temps.
3. Pour chaque paire (problème, algorithme), enregistrez :
    * Le nombre de nœuds explorés.
    * La longueur du chemin trouvé entre la racine et le nœud objectif.
    * Le temps d'exécution. Pour mesurer le temps d'exécution, vous pouvez importer time et utiliser :
        * *start_time = time.time()*
        * *\# résoudre le problème*
        * *end_time = time.time()*
        * *execution_time = end_time - start_time*
4. Enregistrez les résultats dans un dictionaire.



In [24]:
import time

def compare_algorithms(initial_states, goal_state, algorithms, max_it):
    """
    对比多个搜索算法在不同初始状态下的性能表现。

    参数说明：
    - initial_states: 初始状态列表（每个状态都是拼图的二维列表）。
    - goal_state: 目标状态（二维列表）。
    - algorithms: 要测试的算法列表（例如：[breadth_first_search, depth_first_search]）。
    - max_it: 每个算法允许的最大迭代次数（用于限制搜索的规模）。

    返回值：
    - 一个列表results，其中每个元素对应一个算法的所有测试结果。
    """

    # 初始化一个空结果列表，每个算法对应一个子列表
    results = [[] for _ in range(len(algorithms))]

    # 对每个提供的初始状态进行测试
    for initial_state in initial_states:
        print('Solving state')
        visualize_state(initial_state)  # 可视化初始状态，便于直观理解

        # 根据当前初始状态创建拼图问题实例
        problem = Taquin(initial_state, goal_state, len(initial_state[0]))

        # 依次执行所有待测试的搜索算法
        for k in range(len(algorithms)):
            algorithm = algorithms[k]  # 获取当前要运行的算法
            print('Running', algorithm.__name__)  # 打印当前执行的算法名称

            # 记录算法执行前的时间
            start_time = time.time()

            # 执行搜索算法，返回解决方案节点及探索次数
            solution_node, num_explorations = algorithm(problem, max_it)

            # 算法执行后的结束时间
            end_time = time.time()

            # 计算算法执行的总时间（秒）
            execution_time = end_time - start_time

            # 若找到解决方案，记录路径长度（动作数）；若未找到则标记为-1
            if solution_node:
                path_length = len(solution_node.solution())
            else:
                path_length = -1  # 表示未找到解法

            # 记录该算法对当前初始状态的性能结果
            results[k].append({
                "initial_state": initial_state,      # 当前测试的初始状态
                "algorithm": algorithm.__name__,     # 算法名称（如 'breadth_first_search'）
                "num_explorations": num_explorations,# 搜索过程中探索的节点数
                "path_length": path_length,          # 解法路径长度（若无解则为-1）
                "execution_time": execution_time     # 算法执行所需时间
            })

    # 返回所有算法在所有初始状态下的性能结果
    return results


### **Exercice 8**
Comparez les algorithmes BFS et DFS dans les instances suivantes du jeu de Taquin 3x3.
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [ ]:
# 定义一系列不同难度的初始拼图状态（用于评估搜索算法的性能）
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],  # 空白方块位于中间位置，较简单
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],  # 空白左侧偏下，略复杂
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],  # 空白靠上且位置混乱，更复杂
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],  # 非常复杂的状态，可能难解
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]   # 复杂度较高状态
]

# 定义拼图的目标状态（3x3拼图的标准完成状态）
goal_state = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 0]]  # 空白方块最终在右下角位置

# 定义需要比较的搜索算法列表
algorithms = [breadth_first_search, depth_first_search]

# 调用之前定义好的 compare_algorithms 函数进行性能对比：
# - 对于每个初始状态，用BFS与DFS进行搜索
# - 最大搜索次数（max_it）设定为20000，以防止无限循环
results = compare_algorithms(initial_states, goal_state, algorithms, max_it=20000)

# 分别提取并保存BFS和DFS的结果，以便后续分析或展示
results_BFS = results[0]  # 广度优先搜索（BFS）的所有测试结果
results_DFS = results[1]  # 深度优先搜索（DFS）的所有测试结果

# 程序运行完毕后的提示，表明所有测试均已完成
print('Finished')


Solving state


1,2,3
4,5,
6,7,8


Running breadth_first_search
Running depth_first_search


#### Utilisez le code ci-dessous pour visualiser vos résultats

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
# 注意: 需要提前安装tabulate库 (pip install tabulate)
# 如果未安装，可用简单的 print(df) 替代

# 初始化一个空列表，用于存放算法性能数据
Data = []

# 对所有测试过的初始状态，依次提取每个算法的性能数据
for i in range(len(initial_states)):

    # 每个初始状态对应的数据：
    # - State编号（从1开始）
    # - BFS算法探索节点数
    # - DFS算法探索节点数
    # - BFS找到的路径长度（解法步数，-1表示未找到解法）
    # - DFS找到的路径长度（同上）
    # - BFS执行时间（秒，保留2位小数）
    # - DFS执行时间（秒，同上）
    Data.append([
        int(i+1),
        results_BFS[i]['num_explorations'],
        results_DFS[i]['num_explorations'],
        results_BFS[i]['path_length'],
        results_DFS[i]['path_length'],
        round(results_BFS[i]['execution_time'], 2),
        round(results_DFS[i]['execution_time'], 2)
    ])

# 将列表Data转换为pandas数据框（DataFrame），并添加清晰的列标题
df = pd.DataFrame(Data, columns=[
    "State",                  # 状态序号
    "Nodes explored BFS",     # BFS算法探索的节点总数
    "Nodes explored DFS",     # DFS算法探索的节点总数
    "Path length BFS",        # BFS解法路径长度（若未找到解则为-1）
    "Path length DFS",        # DFS解法路径长度（同上）
    "Execution Time BFS",     # BFS执行所花时间（秒）
    "Execution Time DFS"      # DFS执行所花时间（秒）
])

# 使用tabulate库，以美观易读的表格形式输出数据框df
print(tabulate(df, headers='keys', tablefmt='pretty'))

# 若tabulate未安装，可以用 print(df) 简单替代：
# print(df)


# **Extra. Tous les jeux de taquin ont-ils une solution ?**

En cours, nous avons dit que l'espace d'état a une taille de 9!. En réalité, seulement la moitié des configurations sont résolvables. Consultez ce [**lien**](https://fr.wikipedia.org/wiki/Taquin#Configurations_solubles_et_insolubles) ainsi que la fonction suivante pour pour davantage d'*insights*.

In [ ]:
import random

def is_solvable(puzzle):
    """
    检查一个3x3的Taquin拼图状态是否有解（solvable）。

    参数:
    puzzle: 一个3x3二维列表，代表拼图状态。其中0表示空白方块。

    返回值:
    True - 表示该拼图状态有解；
    False - 表示该拼图状态无解。
    """

    # 将拼图的二维状态展平成一维列表（跳过空白方块0）
    flattened = [tile for row in puzzle for tile in row if tile != 0]

    # 计算拼图状态的逆序数（inversions）
    inversions = sum(
        1
        for i in range(len(flattened))              # 对于列表中每个元素tile[i]
        for j in range(i + 1, len(flattened))       # 检查tile[i]右侧的每个元素tile[j]
        if flattened[i] > flattened[j]              # 如果出现tile[i] > tile[j]，即为一个逆序对
    )

    # 根据Taquin拼图规则，若逆序数为偶数，则有解；否则无解。
    return inversions % 2 == 0
